In [11]:
from collections import OrderedDict
from collections import Counter
import numpy as np
import pandas 
import heapq

In [21]:
visit_popularity = Counter()
purchase_popularity = Counter()

with open('train.txt', 'r') as f:
    for line in f.xreadlines():
        visits, purchases = line.strip().split(';')
        for id in visits.split(','):
            visit_popularity[id] += 1
        for id in purchases.split(','):
            purchase_popularity[id] += 1
del purchase_popularity['']

In [22]:
print visit_popularity.most_common(5)
print purchase_popularity.most_common(5)
print sum(visit_popularity.values())
print sum(purchase_popularity.values())

[('73', 677), ('158', 641), ('204', 396), ('262', 387), ('162', 318)]
[('158', 14), ('204', 12), ('3324', 11), ('73', 11), ('977', 10)]
356177
5374


In [15]:
def recommend_by_purchase(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: purchase_popularity.get(x, 0))

def recommend_by_visit(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: visit_popularity.get(x, 0))

In [16]:
def calculate_metrics(recommend, max_count, sessions_file):
    with open(sessions_file, 'r') as f:
        avg_recall = np.zeros(max_count)
        avg_precision = np.zeros(max_count)
        sessions_count = 0
        for line in f.xreadlines():
            visits, purchases = line.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = purchases.split(',')
                for k in xrange(0, max_count):
                    rec = recommend(visits, k+1)
                    avg_recall[k] += len(set.intersection(*[set(rec), set(purchases)]))*1.0/len(purchases)
                    avg_precision[k] += len(set.intersection(*[set(rec), set(purchases)]))*1.0/max_count
                sessions_count+=1
                
    return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': [round(x, 2) for x in avg_recall / float(sessions_count)],
            'avg_precision@k': [round(x, 2) for x in avg_precision / float(sessions_count)]
    }).set_index('k')


In [17]:
calculate_metrics(recommend_by_purchase, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.16,0.69
2,0.21,0.84
3,0.23,0.89
4,0.25,0.91
5,0.25,0.93


In [18]:
calculate_metrics(recommend_by_purchase, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.11,0.46
2,0.15,0.64
3,0.18,0.73
4,0.20,0.79
5,0.21,0.82


In [19]:
calculate_metrics(recommend_by_visit, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.10,0.44
2,0.15,0.63
3,0.18,0.73
4,0.20,0.79
5,0.21,0.82


In [20]:
calculate_metrics(recommend_by_visit, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.10,0.42
2,0.14,0.60
3,0.17,0.70
4,0.19,0.76
5,0.20,0.80


In [ ]:
type(recommend_by_purchase([1,2,3,4,5,1,5,2,2],2)[0])